# Initialize Notebook

## Import packages

In [1]:
import pyspark
import dxdata
import dxpy
import pandas as pd
import json
import requests
import time
import subprocess
import os

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)


## Initialize helper functions

In [2]:
def fetch_gene_info_ensembl(gene_names, species="human", genome_version="GRCh38", batch_size=50):
    gene_info_list = []
    server = "https://rest.ensembl.org"
    endpoint = f"/lookup/symbol/{species}"
    headers = {"Content-Type": "application/json"}
 
    def make_request(batch):
        response = requests.post(server + endpoint, headers=headers, json={"symbols": batch})
        if response.ok:
            return response.json()
        else:
            print(f"Fetching failed for batch: {batch}, Status Code: {response.status_code}")
            return {}
 
    for i in range(0, len(gene_names), batch_size):
        print(f"Processing batch {i // batch_size + 1} / {len(gene_names) // batch_size + 1}")
        batch = gene_names[i:i + batch_size]
        data = make_request(batch)
 
        for gene_name, gene_data in data.items():
            if "seq_region_name" in gene_data:
                gene_info_list.append({
                    "gene_name": gene_data.get("display_name", gene_name),
                    "chromosome": gene_data["seq_region_name"],
                    "start": int(gene_data["start"]),
                    "end": int(gene_data["end"]),
                    "genome_version": genome_version
                })
        time.sleep(0.5)
    return gene_info_list


## Initialize variables

In [3]:
results_dir = "/results"

! dx find projects --name "chorea_wgs" > projectid.txt
projectid = open("projectid.txt", "r")
projectid = projectid.read()
projectid = projectid.split(" : ")[0]


## Download files we need

In [4]:
! dx download /data/ukbb_imputed_genotypes_umap_linearsvc_predicted_labels.txt --overwrite
! dx download /data/chorea_gene_names.txt --overwrite


[===========================================================>] Completed 9,741,159 of 9,741,159 bytes (100%) /opt/notebooks/ukbb_imputed_genotypes_umap_linearsvc_predicted_labels.txtxt
[===========================================================>] Completed 10,052 of 10,052 bytes (100%) /opt/notebooks/chorea_gene_names.txtxt


## Fetch gene info

In [ ]:
with open("chorea_gene_names.txt", "r") as file:
    gene_list = file.read().splitlines()

gene_info_list = fetch_gene_info_ensembl(gene_names=gene_list)
df_gene_info = pd.DataFrame(gene_info_list)
df_gene_info["chromosome"] = pd.Categorical(
    df_gene_info["chromosome"],
    categories=[str(i) for i in range(1, 23)] + ["X", "Y", "MT"],
    ordered=True,
)
df_gene_info = df_gene_info.sort_values(by=["chromosome", "start"]).reset_index(drop=True)


## Combine overlapping genes

In [ ]:
merged = []
current = df_gene_info.iloc[0].copy()
for i in range(1, len(df_gene_info)):
    row = df_gene_info.iloc[i]
    if current["chromosome"] == row["chromosome"] and current["end"] >= row["start"]:
        current["end"] = max(current["end"], row["end"])
        current["start"] = min(current["start"], row["start"])
        current["gene_name"] += " + " + row["gene_name"]
    else:
        merged.append(current)
        current = row.copy()

merged.append(current)
df_gene_info = pd.DataFrame(merged)
display(df_gene_info)


## Combine rows with overlapping WGS files

In [ ]:
df_b_vals = df_gene_info.copy()
df_b_vals.loc[:, "b_start"] = df_b_vals["start"] // 20000 - 1
df_b_vals.loc[:, "b_end"] = df_b_vals["end"] // 20000 + 2
df_b_vals = df_b_vals[["chromosome", "b_start", "b_end"]]

merged = []
current = df_b_vals.iloc[0].copy()
for i in range(1, len(df_b_vals)):
    row = df_b_vals.iloc[i]
    if current["chromosome"] == row["chromosome"] and current["b_end"] >= row["b_start"]:
        current["b_end"] = max(current["b_end"], row["b_end"])
        current["b_start"] = min(current["b_start"], row["b_start"])
    else:
        merged.append(current)
        current = row.copy()

merged.append(current)
df_b_vals = pd.DataFrame(merged)
display(df_b_vals)


## Save gene info

In [ ]:
df_gene_info.to_csv("gene_info.txt", index=False, sep="\t")
df_b_vals.to_csv("b_val_ranges.txt", index=False, sep="\t")

! dx upload gene_info.txt --path /results/gene_info.txt
! dx upload b_val_ranges.txt --path /results/b_val_ranges.txt


# Fetch cohorts

## Grab participant data

In [5]:
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]


## Retrieve Cases

In [6]:
field_names = [
    "eid", "p31", "p34", "p21022", "p131274", "p131012", "p40000_i0",
    "p131016", "p131018", "p131020", "p131022", "p131024", "p131026", 
    "p131028", "p131030", "p131036", "p131038", "p131040", "p131042", 
    "p131046", "p131056", "p131058", "p131062", "p131066", "p131068", 
    "p131070", "p131074", "p131076", "p131078", "p131080", "p131082", 
    "p131084", "p131086", "p131088", "p131090", "p131092", "p131094", 
    "p131096", "p131098", "p131100", "p131102", "p131104", "p131106", 
    "p131108", "p131110", "p131112", "p131114", "p131116", "p131120", 
    "p131122", "p131124", "p131126", "p42018", "p42020", "p42022", 
    "p42024", "p42028", "p42030", "p42032", "p42034", "p42036", 
]
df_cases = participant.retrieve_fields(names=field_names, coding_values="replace", engine=dxdata.connect())
df_cases = df_cases.toPandas()


/opt/conda/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


In [7]:
df_cases.rename(columns={
    "eid":"ID",
    "p31":"GENETIC_SEX", 
    "p34":"BIRTH_YEAR", 
    "p21022":"AGE_OF_RECRUIT",
    "p131274":"CHOREA_DATE",
    "p131012":"HUNTINGTON_DATE",
    "p40000_i0":"DATE_OF_DEATH",
}, inplace=True)
df_cases["ID"] = pd.to_numeric(df_cases["ID"])


In [8]:
df_chorea = df_cases[~df_cases[f"CHOREA_DATE"].isna()]
df_chorea["AGE"] = pd.to_datetime(df_chorea["CHOREA_DATE"]).dt.year - df_chorea["BIRTH_YEAR"]

df_ancestries = pd.read_csv("ukbb_imputed_genotypes_umap_linearsvc_predicted_labels.txt", sep="\t")
df_ancestries.rename(columns={"label":"ancestry", "IID":"ID"}, inplace=True)
df_ancestries = df_ancestries[["ID","ancestry"]]
df_chorea = df_chorea.merge(df_ancestries, on="ID")

ids_chorea = df_chorea["ID"].tolist()


/tmp/ipykernel_1191/3094394521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chorea["AGE"] = pd.to_datetime(df_chorea["CHOREA_DATE"]).dt.year - df_chorea["BIRTH_YEAR"]


## Check for related individuals

In [9]:
! dx download '/Bulk/Genotype\ Results/Genotype\ calls/ukb_rel.dat' --overwrite
df_full_related = pd.read_csv('ukb_rel.dat', sep = ' ')
df_full_related = df_full_related[df_full_related['Kinship'] > 0.0884]

df_related_cohort = df_full_related.loc[df_full_related['ID1'].isin(ids_chorea) & df_full_related['ID2'].isin(ids_chorea)]
df_related_cohort.reset_index(drop=True, inplace=True)
print(f"There are {'NO ' if len(df_related_cohort) == 0 else ''}related individuals!")


[===========================================================>] Completed 3,805,958 of 3,805,958 bytes (100%) /opt/notebooks/ukb_rel.datat
There are NO related individuals!


## Save the IDs of each participant to a txt file

In [ ]:
with open('ids_pre_vcf.txt', 'w') as file:
    for iid in ids_chorea:
        file.write(f"{iid}\n")
        

# Filter out participants without WGS data

## Find participants without WGS data

In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='/Bulk/DRAGEN\ WGS/DRAGEN\ population\ level\ WGS\ variants,\ pVCF\ format\ [500k\ release]/chr1/ukb24310_c1_b1_v1.vcf.gz' "
cmd += f"-iin='/Bulk/DRAGEN\ WGS/DRAGEN\ population\ level\ WGS\ variants,\ pVCF\ format\ [500k\ release]/chr1/ukb24310_c1_b1_v1.vcf.gz.tbi' "
cmd += f"-icmd='bcftools query -l ukb24310_c1_b1_v1.vcf.gz > pvcf_full_ids.txt' "
cmd += f"--instance-type mem1_hdd1_v2_x2 "
cmd += f"--destination '{projectid}:{results_dir}'"

subprocess.run(
    cmd, 
    shell=True, 
)


In [ ]:
"""
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------ PAUSE HERE UNTIL PREVIOUS STEP COMPLETES ------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
"""

## Filter ID lists and clinical data to only include participants with WGS data

In [10]:
! dx download {results_dir}/pvcf_full_ids.txt --overwrite
! grep -Fwf pvcf_full_ids.txt ids_pre_vcf.txt > filtered_sample_ids.txt
! dx upload filtered_sample_ids.txt --path {results_dir}/sample_ids.txt


[===========================================================>] Completed 3,924,328 of 3,924,328 bytes (100%) /opt/notebooks/pvcf_full_ids.txtxt
grep: ids_pre_vcf.txt: No such file or directory


In [13]:
df_chorea[df_chorea["ID"].isin([1900895, 1048019, 5270799])].to_csv("participants_with_variants.txt", sep="\t", index=False)


In [ ]:
with open('filtered_sample_ids.txt', 'r') as file:
    ids_chorea = [int(line.strip()) for line in file]
df_chorea = df_chorea[df_chorea["ID"].isin(ids_chorea)]
display(df_chorea)


## Save final data table

In [ ]:
df_chorea.to_csv(f'chorea_cases.txt', header=True, index=False, sep="\t")
! dx upload chorea_cases.txt --path {results_dir}/chorea_cases.txt


# Fetch pVCF chunks for each gene of interest

In [ ]:
for _, row in df_b_vals.iterrows():
    start_bval = row["b_start"]
    end_bval = row["b_end"]
    chrom = row['chromosome']
    
    for b_val in range(start_bval, end_bval + 1):
        cmd = f"dx run swiss-army-knife "
        cmd += f"-iin='/Bulk/DRAGEN\ WGS/DRAGEN\ population\ level\ WGS\ variants,\ pVCF\ format\ [500k\ release]/chr{chrom}/ukb24310_c{chrom}_b{b_val}_v1.vcf.gz' "
        cmd += f"-iin='/Bulk/DRAGEN\ WGS/DRAGEN\ population\ level\ WGS\ variants,\ pVCF\ format\ [500k\ release]/chr{chrom}/ukb24310_c{chrom}_b{b_val}_v1.vcf.gz.tbi' "
        cmd += f"-iin='{results_dir}/sample_ids.txt' "
        cmd += f"-icmd='bcftools view -O z -S sample_ids.txt ukb24310_c{chrom}_b{b_val}_v1.vcf.gz -o chr{chrom}_b{b_val}.vcf.gz' "
        cmd += f"--instance-type mem2_ssd1_v2_x2 "
        cmd += f"--destination '{projectid}:{results_dir}/01_pvcf_chunks'"

        result = subprocess.run(
            cmd, 
            shell=True, 
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )

        if result.returncode != 0:
            print(f"Error running command for chr = {chrom} b_val = {b_val}:")
            print(result.stderr.decode("utf-8"))


# Combine pVCF chunks

In [ ]:
"""
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------ PAUSE HERE UNTIL PREVIOUS STEP COMPLETES ------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
"""

## Use batches of 1000 files at first

In [ ]:
cmd = f"dx run swiss-army-knife "
icmd = "-icmd='bcftools concat -O z "
num_files = 0

for _, row in df_b_vals.iterrows():
    start_bval = row["b_start"]
    end_bval = row["b_end"]
    chrom = row['chromosome']
    
    for b_val in range(start_bval, end_bval + 1):
        cmd += f"-iin='{results_dir}/01_pvcf_chunks/chr{chrom}_b{b_val}.vcf.gz' "
        icmd += f"chr{chrom}_b{b_val}.vcf.gz "
        num_files += 1
        if num_files % 1000 == 0:
            icmd += f"-o concat_{num_files//1000}.vcf.gz' "
            cmd += icmd
            cmd += f"--instance-type mem2_ssd1_v2_x32 "
            cmd += f"--destination '{projectid}:{results_dir}/02_pvcf_concat'"
            
            result = subprocess.run(
                cmd, 
                shell=True, 
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
            )

            if result.returncode != 0:
                print(f"Error running command:")
                print(result.stderr.decode("utf-8"))
            
            cmd = f"dx run swiss-army-knife "
            icmd = "-icmd='bcftools concat -O z " 

if num_files % 1000 != 0:
    icmd += f"-o concat_{num_files//1000 + 1}.vcf.gz' "
    cmd += icmd
    cmd += f"--instance-type mem2_ssd1_v2_x32 "
    cmd += f"--destination '{projectid}:{results_dir}/02_pvcf_concat'"

    result = subprocess.run(
        cmd, 
        shell=True, 
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )

    if result.returncode != 0:
        print(f"Error running command:")
        print(result.stderr.decode("utf-8"))


## Concat batches together

In [ ]:
cmd = f"dx run swiss-army-knife "
icmd = "-icmd='bcftools concat -O z "
for i in range(1, 13):
    cmd += f"-iin='{results_dir}/02_pvcf_concat/concat_{i}.vcf.gz' "
    icmd += f"concat_{i}.vcf.gz "
icmd += "-o concat.vcf.gz' "
cmd += icmd
cmd += f"--instance-type mem2_ssd1_v2_x32 "
cmd += f"--destination '{projectid}:{results_dir}/02_pvcf_concat'"

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


# Normalize VCF before annotation

### Split multiallelic sites into biallelic records

In [ ]:
"""
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------ PAUSE HERE UNTIL PREVIOUS STEP COMPLETES ------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
"""

In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='{results_dir}/02_pvcf_concat/concat.vcf.gz' "
cmd += f"-icmd='bcftools norm -m-both -O z -o biallelic.vcf.gz concat.vcf.gz' "
cmd += f"--instance-type mem2_ssd1_v2_x16 "
cmd += f"--destination '{projectid}:{results_dir}/03_pvcf_normalized'"

result = subprocess.run(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))
    

### Left-align and normalize

In [ ]:
"""
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------ PAUSE HERE UNTIL PREVIOUS STEP COMPLETES ------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
"""

In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='{results_dir}/03_pvcf_normalized/biallelic.vcf.gz' "
cmd += f"-iin='/data/Homo_sapiens_assembly38.fasta' "
cmd += f"-icmd='bcftools norm -f Homo_sapiens_assembly38.fasta -O z -o normalized.vcf.gz biallelic.vcf.gz' "
cmd += f"--instance-type mem2_ssd1_v2_x16 "
cmd += f"--destination '{projectid}:{results_dir}/03_pvcf_normalized'"

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


# Check CRAM files for expansions

In [ ]:
%%bash

dx download /data/Homo_sapiens_assembly38.fasta --overwrite
wget https://github.com/Illumina/ExpansionHunter/releases/download/v5.0.0/ExpansionHunter-v5.0.0-linux_x86_64.tar.gz
tar -xvzf ExpansionHunter-v5.0.0-linux_x86_64.tar.gz


In [ ]:
! dx download {results_dir}/chorea_cases.txt --overwrite
df_chorea = pd.read_csv("chorea_cases.txt", sep="\t")
ids = df_chorea["ID"].tolist()

for iid in ids:
    ! dx download /Bulk/DRAGEN\ WGS/Whole\ genome\ CRAM\ files\ \(DRAGEN\)\ [500k\ release]/{str(iid)[:2]}/{iid}_24048_0_0.dragen.cram --overwrite
    ! dx download /Bulk/DRAGEN\ WGS/Whole\ genome\ CRAM\ files\ \(DRAGEN\)\ [500k\ release]/{str(iid)[:2]}/{iid}_24048_0_0.dragen.cram.crai --overwrite
    ! ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter --threads 8 --reads {iid}_24048_0_0.dragen.cram --reference Homo_sapiens_assembly38.fasta --variant-catalog ExpansionHunter-v5.0.0-linux_x86_64/variant_catalog/hg38/variant_catalog.json --output-prefix chorea_{iid}
    ! dx upload chorea_{iid}.vcf --path {results_dir}/00_expansions/chorea_{iid}.vcf
    ! rm {iid}_24048_0_0.dragen.cram
    ! rm {iid}_24048_0_0.dragen.cram.crai
    ! rm chorea_{iid}_realigned.bam
    ! rm chorea_{iid}.vcf
    ! rm chorea_{iid}.json


# Generate plink files

In [ ]:
! dx download {results_dir}/chorea_cases.txt --overwrite
df_sex = pd.read_csv(f'chorea_cases.txt', sep="\t")
df_sex = df_sex[["ID","GENETIC_SEX"]]
df_sex["GENETIC_SEX"] = df_sex["GENETIC_SEX"].replace('Female', 2)
df_sex["GENETIC_SEX"] = df_sex["GENETIC_SEX"].replace('Male', 1)
df_sex.rename(columns={"ID":"#IID", "GENETIC_SEX":"SEX"}, inplace=True)
df_sex.to_csv("sex.txt", sep="\t", index=False)
! dx upload sex.txt --path {results_dir}/sex.txt


In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='{results_dir}/03_pvcf_normalized/normalized.vcf.gz' "
cmd += f"-iin='{results_dir}/sex.txt' "
cmd += f"-icmd='plink2 --vcf normalized.vcf.gz --set-all-var-ids \"chr@:#:\\$r:\\$a\" --update-sex sex.txt --new-id-max-allele-len 1017 --make-pgen --out normalized' "
cmd += f"--instance-type mem2_ssd1_v2_x32 "
cmd += f"--destination '{projectid}:{results_dir}/04_plink'"

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


# Annotation

## Filter VCFs to only include a few participants

In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='{results_dir}/03_pvcf_normalized/normalized.vcf.gz' "
cmd += f"-icmd='bcftools view -O z -G normalized.vcf.gz -o annovar_input.vcf.gz' "
cmd += f"--instance-type mem2_ssd1_v2_x4 "
cmd += f"--destination '{projectid}:{results_dir}/05_annotated'"

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


## Fetch Annovar libraries and reference genome data

In [ ]:
! wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
! tar -xzf annovar.latest.tar.gz
! chmod a+x ./annovar/*.pl
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar refGene annovar/humandb/
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar avsnp151 annovar/humandb/
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar clinvar_20240917 annovar/humandb/
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar dbnsfp47a annovar/humandb/
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar dbnsfp47a_interpro annovar/humandb/
! annovar/annotate_variation.pl -downdb -buildver hg38 -webfrom annovar gnomad41_genome annovar/humandb/
! dx download /data/Homo_sapiens_assembly38.fasta --overwrite
! dx download {results_dir}/05_annotated/annovar_input.vcf.gz


## Perform annotation

In [ ]:
cmd = f"annovar/table_annovar.pl annovar_input.vcf.gz annovar/humandb/ "
cmd += f"--buildver hg38 "
cmd += f"--thread 72 "
cmd += f"--remove "
cmd += f"--protocol refGene,avsnp151,clinvar_20240917,dbnsfp47a,dbnsfp47a_interpro,gnomad41_genome "
cmd += f"--operation g,f,f,f,f,f "
cmd += f"--nopolish "
cmd += f"--nastring . "
cmd += f"--out annotated "
cmd += f"--vcfinput "

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


In [ ]:
! mv annotated.hg38_multianno.txt annotated.txt
! dx upload annotated.txt --path {results_dir}/05_annotated/annotated.txt


# Allele frequencies

In [ ]:
cmd = f"dx run swiss-army-knife "
cmd += f"-iin='{results_dir}/04_plink/normalized.pgen' "
cmd += f"-iin='{results_dir}/04_plink/normalized.pvar' "
cmd += f"-iin='{results_dir}/04_plink/normalized.psam' "
cmd += f"-icmd='plink2 --pfile normalized --freq --out frequencies' "
cmd += f"--instance-type mem2_ssd1_v2_x16 "
cmd += f"--destination '{projectid}:{results_dir}/06_frequencies'"

result = subprocess.run(
    cmd, 
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

if result.returncode != 0:
    print(f"Error running command:")
    print(result.stderr.decode("utf-8"))


# Zygosity

## Recode files

In [ ]:
for chrnum in list(range(1, 23)) + ["X","Y","MT"]:
    cmd = f"dx run swiss-army-knife "
    cmd += f"-iin='{results_dir}/04_plink/normalized.pgen' "
    cmd += f"-iin='{results_dir}/04_plink/normalized.pvar' "
    cmd += f"-iin='{results_dir}/04_plink/normalized.psam' "
    cmd += f"-iin='{results_dir}/06_frequencies/frequencies.afreq' "
    cmd += f"-icmd='plink2 --pfile normalized --read-freq frequencies.afreq --chr {chrnum} --export A --out chr{chrnum}' "
    cmd += f"--instance-type mem2_ssd1_v2_x16 "
    cmd += f"--destination '{projectid}:{results_dir}/07_zygosity'"

    result = subprocess.run(
        cmd, 
        shell=True, 
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )

    if result.returncode != 0:
        print(f"Error running command:")
        print(result.stderr.decode("utf-8"))
